<a href="https://colab.research.google.com/github/cogma/llm-book/blob/main/chapter03/3-6-tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第3章 大規模言語モデルの基礎

## 3.6 トークナイゼーション

### 3.6.1 バイト対符号化

In [1]:
# 単語とその頻度
word_freqs = {
    "たのしい": 6,
    "たのしさ": 2,
    "うつくしい": 4,
    "うつくしさ": 1,
}
# 語彙を文字で初期化
vocab = sorted(set([char for word in word_freqs for char in word]))
# 単語とその分割状態
splits = {word: [char for char in word] for word in word_freqs}

In [2]:
from collections import Counter

def compute_most_frequent_pair(
    splits: dict[str, list[str]]
) -> tuple[str, str]:
    """
    最も頻度の高い隣接するサブワードの組を計算する
    """
    pair_freqs = Counter()  # サブワードの組のカウンタ
    for word, freq in word_freqs.items():  # すべての単語を処理
        split = splits[word]  # 現在の単語の分割状態を取得
        # すべての隣接したサブワードの組を処理
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            # サブワードの組の頻度に単語の頻度を加算
            pair_freqs[pair] += freq
    # カウンタから最も頻度の高いサブワードの組を取得
    pair, _ = pair_freqs.most_common(1)[0]
    return pair

def merge_pair(
    target_pair: tuple[str, str], splits: dict[str, list[str]]
) -> dict[str, list[str]]:
    """
    サブワードの組を結合する
    """
    l_str, r_str = target_pair
    for word in word_freqs:  # すべての単語を処理
        split = splits[word]  # 現在の単語の分割状態を取得
        i = 0
        # すべての隣接したサブワードの組を処理
        while i < len(split) - 1:
            # サブワードの組が結合対象と一致したら結合
            if split[i] == l_str and split[i + 1] == r_str:
                split = split[:i] + [l_str + r_str] + split[i + 2 :]
            i += 1
        splits[word] = split  # 現在の結合状態を更新
    return splits

In [3]:
for step in range(9):
    # 最も頻度の高い隣接するサブワードの組を計算
    target_pair = compute_most_frequent_pair(splits)
    # サブワードの組を結合
    splits = merge_pair(target_pair, splits)
    # 語彙にサブワードの組を追加
    vocab.append(target_pair)

In [4]:
print(vocab)

['い', 'う', 'く', 'さ', 'し', 'た', 'つ', 'の', ('し', 'い'), ('た', 'の'), ('たの', 'しい'), ('う', 'つ'), ('うつ', 'く'), ('うつく', 'しい'), ('し', 'さ'), ('たの', 'しさ'), ('うつく', 'しさ')]


### 3.6.3 日本語の扱い

In [5]:
!pip install transformers[ja,sentencepiece,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.0 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=81fc7e2a82adea6c9b0d65574c19b7a46fb2c72d3885c2d3977c4b9927000b66
  Stored in directory: /root/.cache/pip/wheels/93/8b/55/dd5978a069678c372520847cf84ba2ec539cb41917c00a2206
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7403 sha256=e0cb62e40e17dea5877c0b6ef1da1175474bb4386f3a9332820e986f

In [6]:
from transformers import AutoTokenizer

mbert_tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-multilingual-cased"
)
print(mbert_tokenizer.tokenize("自然言語処理"))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

['自', '然', '言', '語', '処', '理']


In [7]:
print(mbert_tokenizer.tokenize("自然言語処理にディープラーニングを使う"))

['自', '然', '言', '語', '処', '理', 'に', '##ディ', '##ープ', '##ラー', '##ニング', '##を', '使', 'う']


In [8]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
print(xlmr_tokenizer.tokenize("自然言語処理にディープラーニングを使う"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

['▁', '自然', '言語', '処理', 'に', 'ディー', 'プラ', 'ー', 'ニング', 'を使う']


In [9]:
print(xlmr_tokenizer.tokenize("私は日本で生まれました"))

['▁私は', '日本で', '生まれ', 'ました']


In [10]:
print(xlmr_tokenizer.tokenize("本日はよろしくお願いいたします"))

['▁本', '日は', 'よろしくお願いいたします']


In [11]:
bert_ja_tokenizer = AutoTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-v3"
)
print(
    bert_ja_tokenizer.tokenize("自然言語処理にディープラーニングを使う")
)

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

['自然', '言語', '処理', 'に', 'ディープ', 'ラー', '##ニング', 'を', '使う']


In [12]:
print(bert_ja_tokenizer.tokenize("私は日本で生まれました"))

['私', 'は', '日本', 'で', '生まれ', 'まし', 'た']
